In [ ]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
state_dim = 2
action_dim = 1
# NN for the controller
NNController = torch.nn.Sequential(
            torch.nn.Linear(state_dim, 64),
            torch.nn.Tanh(),
            torch.nn.Linear(64, action_dim),
             )

In [ ]:
# Load hyperparameters from saved state dictionary
state_dict = torch.load('PPO_Pendulum_3_0.pth')

actor_pairs = list(state_dict.items())[:4]
NNController[0].weight.data = actor_pairs[0][1]
NNController[0].bias.data = actor_pairs[1][1]
NNController[2].weight.data = actor_pairs[2][1]
NNController[2].bias.data = actor_pairs[3][1]

# Print the neural network architecture
print(NNController)

### Lyapunov function NN

In [ ]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        # self.to(device) 

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        return out

In [ ]:
def f_value(x):
    #Dynamics
    y = []
    G = 9.8  # gravity
    L = 0.5   # length of the pole 
    m = 0.1  # ball mass
    b = 0.1   # friction
    u = NNController(x)
    y = torch.zeros_like(x)
    y[:,0] = x[:,1]
    y[:,1] = (m*G*L*torch.sin(x[:,0])- b*x[:,1] + u[:,0]) / (m*L**2)
    
    return y

## Parameters

In [ ]:
'''
For learning 
'''
N = 500             # sample size
D_in = 2            # input dimension
H1 = 6              # hidden dimension
D_out = 1           # output dimension
torch.manual_seed(10)  
x = torch.Tensor(N, D_in).uniform_(-3, 3)           
x_0 = torch.zeros([1, 2])

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
vars_ = [x1,x2]
G = 9.8
l = 0.5  
m = 0.1
b = 0.1
config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = -0 # typically this is 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
x_ub = [3., 3.]

In [ ]:
# define the controller obtained by PPO
u_w1 = actor_pairs[0][1].numpy()
u_b1 = actor_pairs[1][1].numpy()
u_w2 = actor_pairs[2][1].numpy()
u_b2 = actor_pairs[3][1].numpy()

u_h1 = []

u_z1 = np.dot(vars_,u_w1.T)+u_b1
for n in range(len(u_z1)):
    u_h1.append(tanh(u_z1[n]))

u_NN = np.dot(u_h1,u_w2.T)+u_b2

f_u = [ x2,
        (m*G*l*sin(x1) + u_NN[0] - b*x2) /(m*l**2)]

## Learning and Falsification of Lyapunov function

In [ ]:
def CheckLyapunov(x, f, V, ball_lb, x_ub, config, epsilon):
    
    ball= Expression(0)
    lie_derivative_of_V = Expression(0)

    x1_bound = logical_and(x[0]<=x_ub[0], x[0]>=-x_ub[0])
    x2_bound = logical_and(x[1]<=x_ub[1], x[1]>=-x_ub[1])
    x12_bound = logical_and(x1_bound, x2_bound)
    
    for i in range(len(x)):
        ball += x[i]*x[i]
        lie_derivative_of_V += f[i]*V.Differentiate(x[i])  
    ball_in_bound = logical_and(ball_lb*ball_lb <= ball, x12_bound)
    
    # Constraint: x ∈ Ball → (V(c, x) > 0 ∧ Lie derivative of V <= 0)     
    condition = logical_and(logical_imply(ball_in_bound, V >= 0),
                           logical_imply(ball_in_bound, lie_derivative_of_V <= epsilon))
                           
    return CheckSatisfiability(logical_not(condition),config)

In [ ]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    
    model = Net(D_in,H1, D_out)
    L = []
    i = 0 
    t = 0
    max_iters = 3000
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        x = x.float()
        u = NNController(x)
        V_candidate = model(x)
        X0 = model(x_0)
        f = f_value(x)
        Circle_Tuning = Tune(x)
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x,model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t()),0)

        # With tuning term 
        Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean() + (X0).pow(2)
       
        
        print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk.item())
        optimizer.zero_grad()
        Lyapunov_risk.backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()

        # Falsification
        if i % 10 == 0:
            
            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f_u, V_learn, ball_lb, x_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

In [ ]:
print(f"Verification results for PPO.")

if (result): 
  print(f"Not a Lyapunov function. Found counterexample: ")
  print(result)
else:
  print("Satisfy conditions with eps = ", epsilon)
  print(V_learn, f" is a Lyapunov function.")

print("Verification time =", t)

In [ ]:
from numpy import *
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm

In [ ]:
def Plot3D(X, Y, V, r):
    # Plot Lyapunov functions  
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(X,Y,V, rstride=5, cstride=5, alpha=0.5, cmap=cm.coolwarm)
    ax.contour(X,Y,V,10, zdir='z', offset=0, cmap=cm.coolwarm)
    
    # Plot Valid region computed by dReal
    theta = np.linspace(0,2*np.pi,50)
    xc = r*cos(theta)
    yc = r*sin(theta)
    # ax.plot(xc[:],yc[:],'r',linestyle='--', linewidth=2 ,label='Valid region')
    plt.legend(loc='upper right')
    return ax

def Plotflow(Xd, Yd, t):
    # Plot phase plane 
    DX, DY = f([Xd, Yd],u,t)
    plt.streamplot(Xd,Yd,DX,DY, color=('gray'), linewidth=0.5,
                  density=0.8, arrowstyle='-|>', arrowsize=1)

In [ ]:
x2_ub = 10
X = np.linspace(-3, 3, 100) 
Y = np.linspace(-x2_ub, x2_ub, 100)
x1, x2 = np.meshgrid(X,Y)
# Lyapunov functions
# 0.029
V_PPO = tanh((2.7219865322113037 + 1.9835379123687744 * tanh((-0.83820348978042603 + 0.43679541349411011 * x1 + 0.084628865122795105 * x2)) + 3.1041328907012939 * tanh((0.047195754945278168 - 0.13909927010536194 * x1 + 0.011791212484240532 * x2)) - 0.38476818799972534 * tanh((0.10768817365169525 - 1.282874584197998 * x1 - 1.4927834272384644 * x2)) - 0.81294751167297363 * tanh((0.38774198293685913 + 1.1007462739944458 * x1 + 1.303046703338623 * x2)) - 1.5015192031860352 * tanh((0.85250276327133179 - 0.063904657959938049 * x1 + 0.11575903743505478 * x2)) - 0.1492917537689209 * tanh((1.4842376708984375 - 2.2573776245117188 * x1 - 4.4141445159912109 * x2)))) 

# V_lqr = 85.5250*x1**2 + 2*14.1518*x1*x2 + 2.4269*x2**2 

V_SOL =  ((x1 * (2.14609 * x1 + 0.0255221 * x2 - 1.41095 * sin(x1) - 1.0833e-11 * sin(x2))) + (x2 * (0.0255221 * x1 + 0.0321791 * x2 + 0.0371073 * sin(x1) - 1.32529e-12 * sin(x2))) + (( - 1.41095 * x1 + 0.0371073 * x2 + 1.86132 * sin(x1) + 1.0833e-11 * sin(x2)) * sin(x1)) + (( - 1.0833e-11 * x1 - 1.32529e-12 * x2 + 1.0833e-11 * sin(x1) + 1.32531e-12 * sin(x2)) * sin(x2)))

V_SGA = - 0.000045075921492717859465867435151832*x1**8 - 0.000013820624916825717899355419486694*x1**7*x2 - 0.0000016755039767149002975769574342658*x1**6*x2**2 + 0.0022935503580488372709701168056536*x1**6 + 0.0000000084770378408610811335679789223875*x1**5*x2**3 + 0.00066246630028376451417792942327259*x1**5*x2 - 0.000000090872738160405297060009532782255*x1**4*x2**4 + 0.0000587712257074670581397334495673*x1**4*x2**2 - 0.043293162236620342030779825555*x1**4 + 0.000000048284186762233833650054905751692*x1**3*x2**5 - 0.00000070693399884904665290885692796379*x1**3*x2**3 - 0.011498721971926318237042992902235*x1**3*x2 - 0.000000042220880993703541178778525213859*x1**2*x2**6 + 0.0000024993174736924599113249316433513*x1**2*x2**4 - 0.00062869374168708543749955332395284*x1**2*x2**2 + 1.3766040360257852881175279547738*x1**2 + 0.000000019070032453503035836501227979603*x1*x2**7 - 0.0000010268565695710830158962861455953*x1*x2**5 + 0.0000099899285265325336749039124924384*x1*x2**3 + 0.1328515321125741824744458220011*x1*x2 - 0.0000000171545267437991843328792785435*x2**8 + 0.00000088295176323236077244354601115009*x2**6 - 0.000017928349508919918322384839560953*x2**4 + 0.033164814719539552652809778191405*x2**2

V_ADP = -(8348705711791185*x1**4)/72057594037927936 - (8055519091113769*x1**2*x2**2)/1152921504606846976 + (6394020526263095*x1**2)/4503599627370496  + (4895481769477553*x1*x2)/36028797018963968  + (6438708792788647*x2**4)/2.95147905179e20 + (2371300731736623*x2**2)/72057594037927936

ax = Plot3D(x1,x2,V_SOL,4)
ax.set_xlabel('$\Theta$')
ax.set_ylabel('$\dot{\Theta}$')
ax.set_zlabel('V')
plt.title('Lyapunov Function')
plt.show()

In [ ]:
def f(y,u,t) :
    #parameters
    G = 9.8 
    L = 0.5  
    m = 0.1  
    b = 0.1   
    x1,x2 = y
    dydt =[x2,  (m*G*L*sin(x1) + u.detach().numpy() - b*x2) / (m*L**2)]
    return dydt

ax = plt.gca()

ax.contour(x1,x2,V_SOL-2.83,0,linewidths=2, colors='k')
ax.contour(x1,x2,V_SGA-1.9,0,linewidths=2, colors='b',linestyles=':')
ax.contour(x1,x2,V_ADP-0.052,0,linewidths=2, colors='g',linestyles='-.')
ax.contour(x1,x2,V_PPO-0.56,0,linewidths=2, colors='m',linestyles='--')


ax.contour(x1,x2,V_SOL,8,linewidths=0.4, colors='k')
c1 = ax.contourf(x1,x2,V_SOL,8,alpha=0.4,cmap=cm.coolwarm)
plt.colorbar(c1)

plt.title('Region of Attraction')
plt.legend([plt.Rectangle((0,0),1,2,color='k',fill=False,linewidth = 2),plt.Rectangle((0,0),1,2,color='b',fill=False,linewidth = 2,linestyle=':'),
plt.Rectangle((0,0),1,2,color='g',fill=False,linewidth = 2,linestyle='-.'),plt.Rectangle((0,0),1,2,color='m',fill=False,linewidth = 2,linestyle='--')]\
           ,['SOL','SGA','ADP','PPO_NN'],bbox_to_anchor=(1.05, -0.18), loc=0, borderaxespad=0, ncol = 4 ) 
plt.xlabel('$x_1$ (angle)')
plt.ylabel('$x_2$ (angular velocity)')
plt.savefig('ip_roa.pdf', dpi=600, bbox_inches='tight')
plt.show()